## Load necessary modules

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.
/home/cvlab/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cvlab/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cvlab/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cvlab/.local/lib/python3.6/site-packages/tensorflow/python/fra

## Load RetinaNet model

In [2]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
# model_path = os.path.join('..', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')
model_path = '/mnt/8A2A8B2E2A8B15FB/Ted/models/results/retinanet/train/garb.h5'
# load retinanet model
model = models.load_model(model_path, backbone_name='resnet101')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'garbage'}
# labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/home/cvlab/.local/lib/python3.6/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## Run detection on example

In [3]:
base_path = '/mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage'
folders = ['VOC_Test_Easy','VOC_Test_Hard']
split = 'test' #can be train, train_val or test
savedir = '/mnt/8A2A8B2E2A8B15FB/Ted/models/results/retinanet/predict'

for folder in folders:
    txt_file = os.path.join(base_path,folder,'ImageSets/Main',split + '.txt')
    f = open(txt_file,'r')
    lines = f.readlines()
    for line in lines:
        img_name = line.strip()
        img = os.path.join(base_path,folder,'JPEGImages',img_name + '.jpg')     
        print('testing image ' + img + '\n')
        try:
            image = cv2.imread(img)
        except:
            print(img + ' does not exist')
            continue
        else:
            # copy to draw on
            draw = image.copy()
#             draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

            # preprocess image for network
            image = preprocess_image(image)
            image, scale = resize_image(image)

            # process image
    #         start = time.time()
            boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    #         print("processing time: ", time.time() - start)

            # correct for image scale
            boxes /= scale
            annot = []
            # visualize detections
            for box, score, label in zip(boxes[0], scores[0], labels[0]):
                # scores are sorted so we can break
                if score < 0.5:
                    break

    #             color = label_color(label
                color = (0,0,255)

                b = box.astype(int)
                draw_box(draw, b, color=color)

                caption = "{} {:.2f}".format(labels_to_names[label], score)
    #             print(labels_to_names[label],score)
                annot.append(caption + ' ' + str(b[0])+ ' ' + str(b[1])+ ' ' + str(b[2])+ ' ' + str(b[3]))
            if not os.path.exists(os.path.join(savedir,folder)):
                    os.mkdir(os.path.join(savedir,folder))
            f = open(os.path.join(savedir,folder,img_name +'.txt'),'w+')
            for annotation in annot:
                f.write(annotation + '\n')
            f.close()
#             draw_caption(draw, b,caption)

        cv2.imwrite(os.path.join(savedir,folder,img_name +'.jpg'), draw) 
#         plt.figure(figsize=(15, 15))
#         plt.axis('off')
#         plt.imshow(draw)
#         plt.show()

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im19.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im20.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im21.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im22.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im23.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im24.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im25.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im26.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im27.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im28.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImage

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im300.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im301.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im302.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im303.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im304.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im305.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im306.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im307.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im308.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im309.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im547.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im548.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im549.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im550.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im551.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im552.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im553.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im554.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im555.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im556.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im685.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im686.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im687.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im688.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im689.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im690.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im691.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im692.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im693.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im694.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im927.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im928.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im929.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im930.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im931.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im932.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im933.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im934.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im935.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy/JPEGImages/im936.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Easy

...

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im128.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im129.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im130.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im131.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im132.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im133.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im134.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im135.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im136.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im137.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im217.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im218.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im219.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im220.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im221.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im222.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im223.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im224.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im225.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im226.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im342.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im343.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im344.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im345.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im346.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im347.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im348.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im349.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im350.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im351.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im435.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im436.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im437.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im438.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im439.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im440.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im441.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im442.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im443.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im444.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im524.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im525.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im526.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im527.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im528.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im529.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im530.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im531.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im532.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard/JPEGImages/im533.jpg

testing image /mnt/8A2A8B2E2A8B15FB/Ted/datasets/Garbage/VOC_Test_Hard